In [1]:
# 1. Imports & Paths
import importlib.util
import os, sys, re, pandas as pd
sys.path.append(os.path.join(os.pardir, "utils"))

SAVE_DIR = os.path.join(os.pardir, "data/embeddings")
DATA_DIR = os.path.join(os.pardir, "data")
resume_path = os.path.join(DATA_DIR, "resumes_cleaned.csv")
jobs_path   = os.path.join(DATA_DIR, "jobs_cleaned.csv")

In [2]:
%load_ext autoreload
%autoreload 2
from utils import *


c:\Users\adity\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 2. Load Data
resume_df = pd.read_csv(resume_path)
job_posts_df = pd.read_csv(jobs_path)

In [4]:
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\adity\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adity\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

In [5]:
resume_embeddings = model.encode(resume_df["Resume_clean"].tolist(), show_progress_bar=True)
job_embeddings = model.encode(job_posts_df["job_text_clean"].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 171/171 [04:59<00:00,  1.75s/it]


In [6]:
# --- Save embeddings ---
np.save(os.path.join(SAVE_DIR, "resume_embeddings.npy"), resume_embeddings)
np.save(os.path.join(SAVE_DIR, "job_embeddings.npy"), job_embeddings)
